<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/>
        Importando Bibliotecas
    </strong>
</p>

In [64]:
import pandas as pd
import xlwings as xw

<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/>
        Lendo base de dados
    </strong>
</p>

In [65]:
def ler_arquivo(nome_arquivo):
    try:
        return pd.read_excel(f'bases/{nome_arquivo}.xlsx')
    except FileNotFoundError as e:
        return f'Erro: {e}'

In [66]:
df = ler_arquivo('dados de entregas')

<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/>
        SLA
    </strong>
</p>

In [67]:
# # Jeito 1 - Filtrando dentro do loc

# # Total de entregas sem atraso
# # sla_ok = df.loc[ (df['dias_atraso'] == 0) & 
# #     (df['status'] == 'ENTREGUE')].shape[0] 
# # sla_ok

# # # Total de entregas
# # df_total_entregas = df.loc[ df['status'] == 'ENTREGUE'].shape[0]
# # df_total_entregas

# # # percentual SLA
# # sla_percentual = (sla_ok / df_total_entregas) * 100

# print(f'{round(sla_percentual,2)}% das entregas foram realizadas dentro do prazo!')

In [68]:
# Jeito 2 - usando a tabela criada para total entregas

# Total de entregas
df_total_entregas = df.loc[ df['status'] == 'ENTREGUE']
df_total_entregas

# Total de entregas sem atraso
sla_ok = df_total_entregas.loc[(df_total_entregas['dias_atraso'] == 0)]
sla_ok

# percentual SLA
sla_percentual = (sla_ok.shape[0] / df_total_entregas.shape[0]) * 100

print(f'{round(sla_percentual,2)}% das entregas foram realizadas dentro do prazo!')

37.24% das entregas foram realizadas dentro do prazo!


<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/>
        KPI - Entregas Atrasadas
    </strong>
</p>

In [69]:
# Resultado utilizando o jeito 1

# entregas_atrasadas = df.loc[ 
    # (df['dias_atraso'] > 0) & 
    # (df['status'] == 'ENTREGUE')].shape[0]
# entregas_atrasadas

# sla_percentual_atrasadas = (entregas_atrasadas/df_total_entregas) * 100
# sla_percentual_atrasadas

# print(f'{round(sla_percentual_atrasadas,2)}% das entregas foram realizadas fora do prazo!')

In [70]:
# Resultado utilizando jeito 2

percentual_atraso = (
    df_total_entregas.loc [ df_total_entregas['dias_atraso'] > 0].shape[0] / 
    df_total_entregas.shape[0]
) * 100

round(percentual_atraso,2)

62.76

<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/>
        KPI - Análise por Filial
    </strong>
</p>

In [71]:
# Média de dias de atraso por filial
sla_filial = df_total_entregas.groupby('filial')['dias_atraso'].mean()

round(sla_filial,2)

filial
MG    1.84
PR    1.92
RJ    1.81
RS    1.97
SC    1.91
SP    1.80
Name: dias_atraso, dtype: float64

In [72]:

# lambda cria a função sem precisar nomea-la

In [73]:
# Validando palavra chavre sem trabalhar com dataframe (tabela)
desc = 'python é bom demais, a linguagem mais traquila que existe'

palavras = desc.split()

try:
    palavras.index('Python')
except ValueError as e:
    print(e)

'Python' is not in list


<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/>
        Padronização da Descrição
    </strong>
</p>

In [74]:
df['descricao_entrega'] = (
    df['descricao_entrega']
    .str.lower()
    .str.strip()
)

<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/>
        Separar as entregas atrasadas
    </strong>
</p>

In [75]:
df_atrasos = df.loc[ df['dias_atraso'] > 0 ].copy()

<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/>
        Categorizando as causas de atrasos
    </strong>
</p>

In [76]:
df_atrasos.loc[
    df_atrasos['descricao_entrega'].str.contains('chuva', na=False),
    "causa_atraso"
] = "Clima"


df_atrasos.loc[
    df_atrasos['descricao_entrega'].str.contains('extravio', na=False),
    "causa_atraso"
] = "Extravio"

df_atrasos.loc[
    df_atrasos['descricao_entrega'].str.contains('ausente', na=False),
    "causa_atraso"
] = "Cliente Ausente"

df_atrasos.loc[
    df_atrasos['descricao_entrega'].str.contains('operacional|rota', na=False),
    "causa_atraso"
] = "Problema Operacional"

df_atrasos['causa_atraso'] = df_atrasos['causa_atraso'].fillna('Outros')

df_atrasos['causa_atraso'].head(5)

0           Extravio
2    Cliente Ausente
3    Cliente Ausente
5              Clima
6             Outros
Name: causa_atraso, dtype: object

<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/>
       Principais causas de atrasos
    </strong>
</p>

In [77]:
round(df_atrasos['causa_atraso'].value_counts(normalize=True) * 100,2).reset_index(name='%')

,causa_atraso,%
0,Outros,49.58
1,Extravio,13.37
2,Cliente Ausente,12.67
3,Clima,12.67
4,Problema Operacional,11.71


<p style="font-size: 1.1rem; color:#9beb3b; text-align: center; font-family: monospace ">
    <strong><br/><br/>
       Atrasos por Filial
    </strong>
</p>

In [91]:
causas_por_filiais = df_atrasos.groupby(['filial','causa_atraso']) \
    .size().reset_index(name='qtd')           \
    .sort_values(['filial','qtd'],ascending=[True,False])

total_por_filial = (
    causas_por_filiais.groupby('filial')['qtd'].transform('sum')
)

causas_por_filiais['Percentual'] = (
    causas_por_filiais['qtd'] / total_por_filial
) * 100

causas_por_filiais


,filial,causa_atraso,qtd,Percentual
3,MG,Outros,274,47.404844
1,MG,Clima,84,14.532872
0,MG,Cliente Ausente,83,14.359862
2,MG,Extravio,76,13.148789
4,MG,Problema Operacional,61,10.553633
8,PR,Outros,326,51.419558
6,PR,Clima,86,13.564669
9,PR,Problema Operacional,79,12.460568
7,PR,Extravio,74,11.671924
5,PR,Cliente Ausente,69,10.883281
